In [1]:
import pandas as pd
from typing import *
import json
import numpy as np
from ant_is import run_colony
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

df_arit = pd.read_csv("databases/AG/Aritmética/TreinamentoDesbalanceadoAritPreprocessada.csv", sep=";")
df_esc = pd.read_csv("databases/AG/Escrita/TreinamentoDesbalanceadoEscPreprocessada.csv", sep=";")
df_leit = pd.read_csv("databases/AG/Leitura/TreinamentoDesbalanceadoLeitPreprocessada.csv", sep=";")
print(f"lenghts before drop: Arit: {len(df_arit.index)}, Esc: {len(df_esc.index)} Leit: {len(df_leit.index)}")
df_arit = df_arit.drop_duplicates(ignore_index=True)
df_esc = df_esc.drop_duplicates(ignore_index=True)
df_leit = df_leit.drop_duplicates(ignore_index=True)
print(f"lenghts before After: Arit: {len(df_arit.index)}, Esc: {len(df_esc.index)} Leit: {len(df_leit.index)}")
df_arit

lenghts before drop: Arit: 219, Esc: 222 Leit: 190
lenghts before After: Arit: 193, Esc: 219 Leit: 87


,Catolico,Alergia_alimentar,Acidente,Neurologista,Escolaridade_mae,Pai_casado,Pai_amigado,HF_1grau,Classe_baixa,Classe_media,Classe_alta,Anticoncepcional,Parto,Raven_inferior,Raven_medio,Raven_superior,TDE_MG_Esc,Hiperatividade_mae,Mae_prejuizo_funcional,TDE_MG_Arit
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,Inferior
1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,12,0,Inferior
2,0,0,0,1,2,0,0,0,0,1,1,0,0,1,0,1,1,12,0,Superior
3,0,0,0,1,1,1,1,0,0,1,1,0,0,1,0,1,0,12,0,Inferior
4,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,1,12,0,Superior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,1,0,0,1,1,0,0,0,0,1,1,1,1,0,0,0,0,2,0,Inferior
189,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2,12,0,Inferior
190,1,0,0,1,1,1,0,0,0,1,1,0,1,0,0,0,0,0,0,Inferior
191,1,1,1,0,1,0,0,0,0,1,1,1,0,1,0,0,0,31,1,Inferior


In [2]:

df_valid_arit = pd.read_csv("databases/AG/Aritmética/TesteAritPreprocessada.csv", sep=";")
df_valid_arit = pd.read_csv("databases/AG/Escrita/TesteEscPreprocessada.csv", sep=";")
df_valid_leit = pd.read_csv("databases/AG/Leitura/TesteLeitPreprocessada.csv", sep=";")

df_valid_arit

,Genero,Idade,Catolico,Protestante_Evangelico,Ano_escolar,Problema_audicao,Pouco_peso,Convulsoes_epilepsia,Asma_bronquite,Psicologia,...,TDE_MG_Leit,TDE_MG_Arit,Indulgencia,Punicao,Desatencao_mae,Mae_antes_12_anos,Mae_estado_ansiedade_baixa,Mae_estado_ansiedade_media,Mae_estado_ansiedade_alta,TDE_MG_Esc
0,0,12,0,0,0,0,0,0,0,0,...,0,0,12,8,13,0,0,0,0,Inferior
1,0,12,1,1,1,0,0,1,0,0,...,0,0,12,8,18,0,0,1,1,Inferior
2,0,12,1,0,2,0,1,0,1,1,...,0,0,18,5,5,0,1,1,0,Inferior
3,1,12,1,0,3,0,0,0,0,0,...,0,0,12,8,13,0,0,0,0,Inferior
4,1,12,1,0,3,0,0,0,0,0,...,0,0,12,8,13,0,0,0,0,Inferior
5,1,11,1,0,3,0,0,0,0,0,...,1,1,12,8,13,0,0,0,0,Superior
6,0,11,1,0,3,0,0,0,0,1,...,0,0,8,8,13,0,0,0,0,Inferior
7,0,11,1,0,3,0,0,0,0,0,...,2,1,9,10,13,0,0,0,0,Superior
8,0,11,0,0,3,0,0,0,0,1,...,0,0,12,8,12,0,0,0,0,Inferior
9,0,11,0,0,3,0,0,0,0,1,...,0,0,12,8,16,0,0,0,0,Inferior


In [3]:
CLASSIFICATION_METRICS = ["accuracy", "precision", "recall", "f1-score", "support"]
def get_stratfied_cross_validation_scores(scores: List[Dict]):
    new_score = scores[0].copy()
    keys = new_score.keys()
    test_length = len(scores)
    for key in keys:
        if isinstance(new_score[key], dict):
            for sub_key in new_score[key].keys():
                value = sum([i[key][sub_key] for i in scores]) / test_length
                new_score[key][sub_key] = value
        else:
            value = sum([i[key] for i in scores]) / test_length
            new_score[key] = value

    return new_score


def get_average_results(results, result_key, num_iterations):
    avg_results = {}
    for metric_name in CLASSIFICATION_METRICS[1:]:
        avg_results[metric_name] = sum([results[i][result_key]["macro avg"][metric_name] for i in results.keys()]) / \
                                  num_iterations

    avg_results["accuracy"] = sum([results[i][result_key]["accuracy"] for i in results.keys()]) / num_iterations
    return avg_results

def create_test(classifier,
                num_iterations: int,
                num_folds: int,
                X: np.ndarray,
                y: np.ndarray,
                X_valid,
                y_valid,
                output_name: str,
                reduce_instances: bool = False,
                initial_pheromone: float = None,
                evaporation_rate: float = None,
                Q: float = None, ):
    num_instances = X.shape[0]
    results = {}
    for i in range(num_iterations):
        print(f"beginning of iterarion {i}")
        folds = StratifiedKFold(n_splits=num_folds, shuffle=True)
        partial_scores = []
        valid_scores = []
        selected_indices = []
        for train_indices, test_indices in folds.split(X, y):
            train_set = train_indices
            if reduce_instances:
                reduced_indices = run_colony(X[train_indices], y[train_indices], initial_pheromone, evaporation_rate, Q)
                selected_indices.append(reduced_indices.tolist())
                train_set = reduced_indices

            classifier.fit(X[train_set], y[train_set])
            y_pred = classifier.predict(X[test_indices])
            y_pred_valid = classifier.predict(X_valid)
            score = classification_report(y[test_indices], y_pred, output_dict=True, zero_division=1)
            valid_score = classification_report(y_valid, y_pred_valid, output_dict=True, zero_division=1)
            valid_scores.append(valid_score)
            partial_scores.append(score)

        scores = get_stratfied_cross_validation_scores(partial_scores)
        v_score = get_stratfied_cross_validation_scores(valid_scores)
        print(f"accuracy: {scores['accuracy']} ---- {v_score['accuracy']}")
        for metric_name in scores["macro avg"].keys():
            print(f"{metric_name}: {scores['macro avg'][metric_name]} ---- {v_score['macro avg'][metric_name]}")

        print("-------------------------------------------------------")
        new_result = "i" + str(i)
        results[new_result] = {
            "scores": get_stratfied_cross_validation_scores(partial_scores),
            "valid_scores": get_stratfied_cross_validation_scores(valid_scores),
            "partial_scores": partial_scores,
            "partial_valid_scores": valid_scores,
        }

        if reduce_instances:
            results[new_result]["selected_indices"] = selected_indices

    avg_scores = get_average_results(results, "scores", num_iterations)
    avg_valid_scores = get_average_results(results, "valid_scores", num_iterations)

    results["num_iterations"] = num_iterations
    results["num_folds"] = num_folds
    results["strategy"] = "Strafied Cross Validation"  # For a while is the unique validation method availble in tests
    results["avg_scores"] = avg_scores
    results["avg_valid_scores"] = avg_valid_scores
    print("---==Final test results==---")
    for key in avg_scores.keys():
        print(f"{key}: {avg_scores[key]} ---- {avg_valid_scores[key]}")


    print("Generating output JSON file....")
    json.dump(results, open(output_name, 'w'), indent=2)

In [13]:
df_arit

,index,hash,label,count
0,0,6c9abdab20c6724493d948ae60ab733d2826f4aefcaf6f...,Inferior,0
1,1,0fe12a67bc082588442081b9016b263222d6e60c17a777...,Inferior,0
2,2,3f830bbe53b5743e03b07a70d9dff95f6a4f4a0b29ff4b...,Superior,0
3,3,2f4328cd4e34a17380f61ee734b929c0305631b7b5d4c5...,Inferior,0
4,4,e42f742af73f48db517ed45bddfb663bbfeffbaf4d1bad...,Superior,0
...,...,...,...,...
188,188,6dcc7d7fde9826c3eef7f5b935c5e0cbedeef8e234ce54...,Inferior,0
189,189,eb819a254584f11f51a9bfcc5bed92416cb726e9a31ba2...,Inferior,0
190,190,845169a9c93f3cc9ec6448fec8b8d4595a1a6fe6447b2d...,Inferior,0
191,191,8dfcb477847a5d96bc21c909ce9bf490d6f2e94a35bb45...,Inferior,0
